# 🔍 Pre-Schema Design EDA

**Goal:** Understand data thoroughly BEFORE designing database schema

**Key Questions to Answer:**
1. What are the exact data types and value ranges for each column?
2. What are the primary keys? Are there duplicates?
3. What are the relationships between tables? (FK constraints)
4. What data quality issues exist? (missing, invalid, anomalies)
5. What constraints should we add to the schema?

---

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print('Libraries loaded!')

Libraries loaded!


In [28]:
# Load data from CSV files
DATA_DIR = '../data'

orders_df = pd.read_csv(f'{DATA_DIR}/orders.csv')
crm_df = pd.read_csv(f'{DATA_DIR}/crm.csv')
comments_df = pd.read_csv(f'{DATA_DIR}/order_comments.csv')

print(f'orders.csv    : {orders_df.shape[0]:>12,} rows x {orders_df.shape[1]} cols')
print(f'crm.csv       : {crm_df.shape[0]:>12,} rows x {crm_df.shape[1]} cols')
print(f'comments.csv  : {comments_df.shape[0]:>12,} rows x {comments_df.shape[1]} cols')

orders.csv    :    2,720,059 rows x 5 cols
crm.csv       :    2,720,059 rows x 6 cols
comments.csv  :       93,855 rows x 2 cols


---
## 1️⃣ ORDERS TABLE - Deep Analysis

In [29]:
print('='*70)
print('ORDERS TABLE - Basic Info')
print('='*70)
print(orders_df.info())
print('\n')
print('First 5 rows:')
display(orders_df.head())

ORDERS TABLE - Basic Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2720059 entries, 0 to 2720058
Data columns (total 5 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   order_id         int64 
 1   user_id          int64 
 2   is_otd           int64 
 3   order_date       object
 4   delivery_status  object
dtypes: int64(3), object(2)
memory usage: 103.8+ MB
None


First 5 rows:


,order_id,user_id,is_otd,order_date,delivery_status
0,11169853,1385028,1,2025-03-16,delivered
1,11169855,54227,1,2025-03-16,delivered
2,11169857,30492532,1,2025-03-16,delivered
3,11169858,2411140,1,2025-03-16,delivered
4,11169859,1647659,1,2025-03-16,delivered


In [30]:
# Check: Is order_id unique? (Can it be PRIMARY KEY?)
print('='*70)
print('ORDERS - Primary Key Analysis (order_id)')
print('='*70)

total_rows = len(orders_df)
unique_order_ids = orders_df['order_id'].nunique()
duplicate_count = total_rows - unique_order_ids

print(f'Total rows:        {total_rows:,}')
print(f'Unique order_ids:  {unique_order_ids:,}')
print(f'Duplicates:        {duplicate_count:,}')
print(f'\n✅ order_id CAN be PRIMARY KEY: {duplicate_count == 0}')

# If duplicates exist, show examples
if duplicate_count > 0:
    print('\n⚠️ DUPLICATE order_ids found! Examples:')
    dup_ids = orders_df[orders_df.duplicated(subset=['order_id'], keep=False)]
    display(dup_ids.head(10))

ORDERS - Primary Key Analysis (order_id)
Total rows:        2,720,059
Unique order_ids:  2,720,059
Duplicates:        0

✅ order_id CAN be PRIMARY KEY: True


In [31]:
# Check: user_id analysis
print('='*70)
print('ORDERS - user_id Analysis')
print('='*70)

print(f'Total unique users: {orders_df["user_id"].nunique():,}')
print(f'user_id data type:  {orders_df["user_id"].dtype}')
print(f'user_id min value:  {orders_df["user_id"].min()}')
print(f'user_id max value:  {orders_df["user_id"].max()}')
print(f'Any null user_id:   {orders_df["user_id"].isnull().sum()}')

# Orders per user distribution
orders_per_user = orders_df.groupby('user_id').size()
print(f'\nOrders per user statistics:')
print(orders_per_user.describe())

ORDERS - user_id Analysis
Total unique users: 375,998
user_id data type:  int64
user_id min value:  1
user_id max value:  34259343
Any null user_id:   0

Orders per user statistics:
count   375998.00
mean         7.23
std         40.31
min          1.00
25%          1.00
50%          2.00
75%          7.00
max      23079.00
dtype: float64


In [32]:
# Check: is_otd column - THE -1 MYSTERY!
print('='*70)
print('ORDERS - is_otd Analysis (THE -1 MYSTERY)')
print('='*70)

print(f'Data type: {orders_df["is_otd"].dtype}')
print(f'\nValue counts:')
is_otd_counts = orders_df['is_otd'].value_counts(dropna=False).sort_index()
print(is_otd_counts)

print(f'\nAs percentages:')
is_otd_pct = (is_otd_counts / len(orders_df) * 100).round(2)
for val, pct in is_otd_pct.items():
    print(f'  {val}: {pct}%')

# What does -1 mean? Let's check if there's a pattern
print('\n⚠️ What might -1 mean?')
print('   Hypothesis: -1 might be "unknown" or "not applicable"')
print('   Let\'s check delivery_status for rows where is_otd=-1:')

if -1 in orders_df['is_otd'].values:
    minus1_rows = orders_df[orders_df['is_otd'] == -1]
    print(f'\n   delivery_status for is_otd=-1 rows:')
    print(minus1_rows['delivery_status'].value_counts())

ORDERS - is_otd Analysis (THE -1 MYSTERY)
Data type: int64

Value counts:
is_otd
-1     165936
 0     535039
 1    2019084
Name: count, dtype: int64

As percentages:
  -1: 6.1%
  0: 19.67%
  1: 74.23%

⚠️ What might -1 mean?
   Hypothesis: -1 might be "unknown" or "not applicable"
   Let's check delivery_status for rows where is_otd=-1:

   delivery_status for is_otd=-1 rows:
delivery_status
canceled              71690
delivered             44354
canceled_by_driver     4013
new                    1733
accepted                 69
searching                40
picking                  23
delivering                4
Name: count, dtype: int64


In [33]:
# Check: delivery_status column
print('='*70)
print('ORDERS - delivery_status Analysis')
print('='*70)

print(f'Data type: {orders_df["delivery_status"].dtype}')
print(f'\nUnique values ({orders_df["delivery_status"].nunique()}):')
status_counts = orders_df['delivery_status'].value_counts(dropna=False)
print(status_counts)

print(f'\nMissing values: {orders_df["delivery_status"].isnull().sum():,}')

# Check if is_otd relates to delivery_status
print('\n📊 Crosstab: delivery_status vs is_otd')
crosstab = pd.crosstab(orders_df['delivery_status'], orders_df['is_otd'], margins=True, dropna=False)
display(crosstab)

ORDERS - delivery_status Analysis
Data type: object

Unique values (8):
delivery_status
delivered             2598339
canceled                71696
NaN                     44014
canceled_by_driver       4135
new                      1733
accepted                   70
searching                  40
picking                    28
delivering                  4
Name: count, dtype: int64

Missing values: 44,014

📊 Crosstab: delivery_status vs is_otd


is_otd,-1,0,1,All
delivery_status,,,,
accepted,69,1,0,70.00
canceled,71690,3,3,71696.00
canceled_by_driver,4013,72,50,4135.00
delivered,44354,534959,2019026,2598339.00
delivering,4,0,0,4.00
new,1733,0,0,1733.00
picking,23,3,2,28.00
searching,40,0,0,40.00
NaN,44010,1,3,NaN


In [34]:
# Check: order_date column
print('='*70)
print('ORDERS - order_date Analysis')
print('='*70)

orders_df['order_date_parsed'] = pd.to_datetime(orders_df['order_date'], errors='coerce')

print(f'Original dtype:     {orders_df["order_date"].dtype}')
print(f'Parse errors:       {orders_df["order_date_parsed"].isnull().sum()}')
print(f'\nDate range:')
print(f'  Min date: {orders_df["order_date_parsed"].min()}')
print(f'  Max date: {orders_df["order_date_parsed"].max()}')
print(f'  Span:     {(orders_df["order_date_parsed"].max() - orders_df["order_date_parsed"].min()).days} days')

# Check date format samples
print(f'\nSample date values:')
print(orders_df['order_date'].head(10).tolist())

ORDERS - order_date Analysis
Original dtype:     object
Parse errors:       0

Date range:
  Min date: 2025-03-16 00:00:00
  Max date: 2025-09-12 00:00:00
  Span:     180 days

Sample date values:
['2025-03-16', '2025-03-16', '2025-03-16', '2025-03-16', '2025-03-16', '2025-03-16', '2025-03-16', '2025-03-16', '2025-03-16', '2025-03-16']


---
## 2️⃣ CRM TABLE - Deep Analysis

In [35]:
print('='*70)
print('CRM TABLE - Basic Info')
print('='*70)
print(crm_df.info())
print('\n')
print('First 5 rows:')
display(crm_df.head())

CRM TABLE - Basic Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2720059 entries, 0 to 2720058
Data columns (total 6 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   order_id                         int64  
 1   order_date                       object 
 2   crm_delivery_request_count       int64  
 3   crm_fake_delivery_request_count  int64  
 4   rate_to_shop                     float64
 5   rate_to_courier                  float64
dtypes: float64(2), int64(3), object(1)
memory usage: 124.5+ MB
None


First 5 rows:


,order_id,order_date,crm_delivery_request_count,crm_fake_delivery_request_count,rate_to_shop,rate_to_courier
0,11229207,2025-03-19,1,1,NaN,NaN
1,11229208,2025-03-19,0,0,5.00,NaN
2,11229209,2025-03-19,0,0,5.00,NaN
3,11229210,2025-03-19,0,0,NaN,NaN
4,11229211,2025-03-19,0,0,NaN,NaN


In [36]:
# Check: Is order_id unique in CRM? (1-to-1 relationship?)
print('='*70)
print('CRM - order_id Analysis (FK to orders)')
print('='*70)

total_crm = len(crm_df)
unique_crm_order_ids = crm_df['order_id'].nunique()
duplicate_crm = total_crm - unique_crm_order_ids

print(f'Total CRM rows:       {total_crm:,}')
print(f'Unique order_ids:     {unique_crm_order_ids:,}')
print(f'Duplicate order_ids:  {duplicate_crm:,}')
print(f'\nRelationship type: {"1-to-1" if duplicate_crm == 0 else "1-to-many"}')

if duplicate_crm > 0:
    print('\n⚠️ Duplicate order_ids in CRM! Examples:')
    dup_crm = crm_df[crm_df.duplicated(subset=['order_id'], keep=False)].sort_values('order_id')
    display(dup_crm.head(10))

CRM - order_id Analysis (FK to orders)
Total CRM rows:       2,720,059
Unique order_ids:     2,720,059
Duplicate order_ids:  0

Relationship type: 1-to-1


In [37]:
# Check: Do all CRM order_ids exist in orders table?
print('='*70)
print('CRM - Foreign Key Integrity Check')
print('='*70)

orders_order_ids = set(orders_df['order_id'].unique())
crm_order_ids = set(crm_df['order_id'].unique())

crm_not_in_orders = crm_order_ids - orders_order_ids
orders_not_in_crm = orders_order_ids - crm_order_ids

print(f'CRM order_ids NOT in orders:     {len(crm_not_in_orders):,}')
print(f'Orders order_ids NOT in CRM:     {len(orders_not_in_crm):,}')

if len(crm_not_in_orders) > 0:
    print(f'\n⚠️ Orphan CRM records! Sample order_ids:')
    print(list(crm_not_in_orders)[:10])
else:
    print('\n✅ All CRM order_ids exist in orders table')

CRM - Foreign Key Integrity Check
CRM order_ids NOT in orders:     0
Orders order_ids NOT in CRM:     0

✅ All CRM order_ids exist in orders table


In [38]:
# Check: CRM order_date vs orders order_date
print('='*70)
print('CRM - order_date Comparison with Orders')
print('='*70)

# Merge to compare dates
compare_df = orders_df[['order_id', 'order_date']].merge(
    crm_df[['order_id', 'order_date']], 
    on='order_id', 
    suffixes=('_orders', '_crm')
)

# Are they the same?
same_dates = (compare_df['order_date_orders'] == compare_df['order_date_crm']).sum()
diff_dates = len(compare_df) - same_dates

print(f'Matched rows:    {len(compare_df):,}')
print(f'Same dates:      {same_dates:,}')
print(f'Different dates: {diff_dates:,}')

if diff_dates > 0:
    print('\n⚠️ Dates differ! Examples:')
    display(compare_df[compare_df['order_date_orders'] != compare_df['order_date_crm']].head(10))
else:
    print('\n✅ CRM order_date is REDUNDANT (same as orders.order_date)')
    print('   --> Can be removed from CRM table in schema')

CRM - order_date Comparison with Orders
Matched rows:    2,720,059
Same dates:      2,720,059
Different dates: 0

✅ CRM order_date is REDUNDANT (same as orders.order_date)
   --> Can be removed from CRM table in schema


In [39]:
# Check: CRM complaint counts
print('='*70)
print('CRM - Complaint Counts Analysis')
print('='*70)

for col in ['crm_delivery_request_count', 'crm_fake_delivery_request_count']:
    print(f'\n{col}:')
    print(f'  Min:     {crm_df[col].min()}')
    print(f'  Max:     {crm_df[col].max()}')
    print(f'  Mean:    {crm_df[col].mean():.4f}')
    print(f'  Nulls:   {crm_df[col].isnull().sum()}')
    print(f'  Value distribution (top 10):')
    print(crm_df[col].value_counts().head(10))

CRM - Complaint Counts Analysis

crm_delivery_request_count:
  Min:     0
  Max:     73
  Mean:    0.3412
  Nulls:   0
  Value distribution (top 10):
crm_delivery_request_count
0    2139126
1     367655
2     133628
3      48715
4      18182
5       7497
6       3071
7       1186
8        478
9        207
Name: count, dtype: int64

crm_fake_delivery_request_count:
  Min:     0
  Max:     2
  Mean:    0.0099
  Nulls:   0
  Value distribution (top 10):
crm_fake_delivery_request_count
0    2693178
1      26860
2         21
Name: count, dtype: int64


In [40]:
# Check: Rating columns
print('='*70)
print('CRM - Rating Columns Analysis')
print('='*70)

for col in ['rate_to_shop', 'rate_to_courier']:
    print(f'\n{col}:')
    non_null = crm_df[col].dropna()
    null_count = crm_df[col].isnull().sum()
    null_pct = null_count / len(crm_df) * 100
    
    print(f'  Non-null values: {len(non_null):,} ({100-null_pct:.1f}%)')
    print(f'  Null values:     {null_count:,} ({null_pct:.1f}%)')
    
    if len(non_null) > 0:
        print(f'  Min:  {non_null.min()}')
        print(f'  Max:  {non_null.max()}')
        print(f'  Mean: {non_null.mean():.2f}')
        print(f'\n  Value distribution:')
        print(non_null.value_counts().sort_index())

CRM - Rating Columns Analysis

rate_to_shop:
  Non-null values: 1,126,369 (41.4%)
  Null values:     1,593,690 (58.6%)
  Min:  1.0
  Max:  5.0
  Mean: 4.42

  Value distribution:
rate_to_shop
1.00     75602
2.00     21540
3.00     60020
4.00    161759
5.00    807448
Name: count, dtype: int64

rate_to_courier:
  Non-null values: 714,590 (26.3%)
  Null values:     2,005,469 (73.7%)
  Min:  1.0
  Max:  5.0
  Mean: 4.68

  Value distribution:
rate_to_courier
1.00     27134
2.00      5905
3.00     17523
4.00     65383
5.00    598645
Name: count, dtype: int64


---
## 3️⃣ COMMENTS TABLE - Deep Analysis

In [41]:
print('='*70)
print('COMMENTS TABLE - Basic Info')
print('='*70)
print(comments_df.info())
print('\n')
print('First 10 rows:')
display(comments_df.head(10))

COMMENTS TABLE - Basic Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93855 entries, 0 to 93854
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   order_id     93855 non-null  int64 
 1   description  93855 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.4+ MB
None


First 10 rows:


,order_id,description
0,11170643,باتشکرازشما
1,11171001,چندین بار به این فروشگاه سفارش دادم و همیشه را...
2,11171094,۲ عدد بستنی اضافه ارسال شده
3,11171323,رشته سوپ اشتباه ارسال شده
4,11171466,عالی بود فقط قیمت بالا بود
5,11171521,۴روز از تاریخ تولید شیر گذشته
6,11171630,سلام کیفیت عالی بود واقعا ممنونم خدا بهتون برک...
7,11171661,بجای تک ماکارون ،زر ماکارون ارسال کردن که طعمش...
8,11171740,فروشگاه بلدرچین منجمد ارسال کرده در صورتی که ب...
9,11171798,سفارشم رو اشتباه ارسال کردند.


In [42]:
# Check: Can one order have multiple comments?
print('='*70)
print('COMMENTS - order_id Analysis')
print('='*70)

total_comments = len(comments_df)
unique_comment_order_ids = comments_df['order_id'].nunique()
duplicate_comments = total_comments - unique_comment_order_ids

print(f'Total comments:           {total_comments:,}')
print(f'Unique order_ids:         {unique_comment_order_ids:,}')
print(f'Orders with multi-comment:{duplicate_comments:,}')

if duplicate_comments > 0:
    print(f'\nRelationship: 1-to-MANY (one order can have multiple comments)')
    
    # Show distribution
    comments_per_order = comments_df.groupby('order_id').size()
    print(f'\nComments per order distribution:')
    print(comments_per_order.value_counts().sort_index().head(10))
    
    # Show example of order with multiple comments
    multi_comment_order = comments_per_order[comments_per_order > 1].index[0]
    print(f'\nExample: order_id {multi_comment_order} has {comments_per_order[multi_comment_order]} comments:')
    display(comments_df[comments_df['order_id'] == multi_comment_order])
else:
    print(f'\nRelationship: 1-to-1 (each order has at most one comment)')

COMMENTS - order_id Analysis
Total comments:           93,855
Unique order_ids:         87,618
Orders with multi-comment:6,237

Relationship: 1-to-MANY (one order can have multiple comments)

Comments per order distribution:
1    81441
2     6118
3       58
4        1
Name: count, dtype: int64

Example: order_id 11171606 has 2 comments:


,order_id,description
907,11171606,نون پر کنجد سفارش دادم ولی درواقع روی نون خیلی...
908,11171606,نون پر کنجد سفارش دادم ولی درواقع روی نون خیلی...


In [43]:
comments_per_order[comments_per_order > 2]

order_id
11189741    3
11190755    3
11191722    3
11191982    3
11192341    3
11193401    3
11194658    3
11194659    3
11195807    3
11197133    3
11199758    3
11200598    3
11201326    3
11202275    3
11202796    3
11203571    3
11218598    3
11248922    3
11251824    3
11259569    3
11282154    3
11361544    3
11539598    4
11661630    3
11672146    3
11702168    3
11737350    3
11823672    3
11891055    3
12020884    3
13452641    3
13571717    3
13717489    3
13754503    3
13916611    3
13980922    3
14042194    3
14136137    3
14158825    3
14311311    3
14382519    3
14403524    3
14407366    3
14416468    3
14449885    3
14474289    3
14493368    3
14497646    3
14537555    3
14620556    3
14713640    3
14744843    3
14763279    3
14764670    3
14767168    3
14768425    3
14890276    3
14972381    3
14980082    3
dtype: int64

In [44]:
    # Show example of order with multiple comments
    for i in range(10):
        multi_comment_order = comments_per_order[comments_per_order > 2].index[i]
        print(f'\nExample: order_id {multi_comment_order} has {comments_per_order[multi_comment_order]} comments:')
        display(comments_df[comments_df['order_id'] == multi_comment_order])


Example: order_id 11189741 has 3 comments:


,order_id,description
393,11189741,نان مزه ماندگی میداد و تازه نبود.
394,11189741,نان مزه ماندگی میداد و تازه نبود.
395,11189741,نان مزه ماندگی میداد و تازه نبود.



Example: order_id 11190755 has 3 comments:


,order_id,description
419,11190755,سلام و روز بخیر در ارسال همه اقلام سفارش دقت ش...
420,11190755,سلام و روز بخیر در ارسال همه اقلام سفارش دقت ش...
421,11190755,سلام و روز بخیر در ارسال همه اقلام سفارش دقت ش...



Example: order_id 11191722 has 3 comments:


...

,order_id,description
534,11194659,با سلام با توجه به نداشتن کیفیت خرما رو تحویل ...
535,11194659,با سلام با توجه به نداشتن کیفیت خرما رو تحویل ...
536,11194659,با سلام با توجه به نداشتن کیفیت خرما رو تحویل ...



Example: order_id 11195807 has 3 comments:


,order_id,description
567,11195807,بعد از ۱.۵ ساعت معطلی بار تحویل داده نشد.بسیار...
568,11195807,بعد از ۱.۵ ساعت معطلی بار تحویل داده نشد.بسیار...
569,11195807,بعد از ۱.۵ ساعت معطلی بار تحویل داده نشد.بسیار...



Example: order_id 11197133 has 3 comments:


,order_id,description
606,11197133,سلام کره 50 گرمی مهگل نیامده لطفاً رسیدگی کنید...
607,11197133,سلام کره 50 گرمی مهگل نیامده لطفاً رسیدگی کنید...
608,11197133,سلام کره 50 گرمی مهگل نیامده لطفاً رسیدگی کنید...


In [45]:
# Check: FK integrity - Do all comment order_ids exist in orders?
print('='*70)
print('COMMENTS - Foreign Key Integrity Check')
print('='*70)

comment_order_ids = set(comments_df['order_id'].unique())
comments_not_in_orders = comment_order_ids - orders_order_ids

print(f'Comment order_ids NOT in orders: {len(comments_not_in_orders):,}')
print(f'Percentage of orphans: {len(comments_not_in_orders)/len(comment_order_ids)*100:.2f}%')

if len(comments_not_in_orders) > 0:
    print(f'\n⚠️ ORPHAN COMMENTS FOUND!')
    print(f'   These comments reference orders that don\'t exist in orders.csv')
    print(f'\n   Sample orphan order_ids: {list(comments_not_in_orders)[:20]}')
    
    # Show sample orphan comments
    orphan_comments = comments_df[comments_df['order_id'].isin(comments_not_in_orders)]
    print(f'\n   Sample orphan comments:')
    display(orphan_comments.head(10))

COMMENTS - Foreign Key Integrity Check
Comment order_ids NOT in orders: 5,037
Percentage of orphans: 5.75%

⚠️ ORPHAN COMMENTS FOUND!
   These comments reference orders that don't exist in orders.csv

   Sample orphan order_ids: [15138816, 15171584, 15040530, 15040536, 11403289, 15106075, 14975008, 15138857, 14975023, 15138872, 15040572, 11403324, 15171648, 14909512, 15171659, 15007832, 15040603, 15040605, 14942303, 15040623]

   Sample orphan comments:


,order_id,description
6554,11391040,کسربودن اقلام.عدم پرداخت مبلغ کالایی که کم بود!
6555,11391106,تاریخ انقضا ی ماست ۲۰ فروردین هست.دوتا سطل هم ...
6556,11391155,بدترین نانی که تا به امروز تحویل گرفتم . پس از...
6557,11391159,بعد از حدود سه ساعت ارسال انجام شد، با اینکه خ...
6558,11391172,فقط سفارش می‌گیرند ولی برای تحویل سفارش هیچ اق...
6559,11391185,جرمگیر یک ساله تاریخش گذشته و تازه همونم قیمت ...
6560,11391187,پرتقال خوب نبود
6561,11391199,ممنون
6562,11391221,از دفعه قبل خیلی بهتره فقط ی مقدار خمیر بود\nد...
6563,11391283,کیت کت بجای ۹ عدد ۲ تا ارسال شد پیگیری شود


In [46]:
# Check: Description column (text analysis)
print('='*70)
print('COMMENTS - Description Text Analysis')
print('='*70)

# Null/empty check
null_desc = comments_df['description'].isnull().sum()
empty_desc = (comments_df['description'] == '').sum()

print(f'Null descriptions:  {null_desc}')
print(f'Empty descriptions: {empty_desc}')

# Text length distribution
comments_df['desc_length'] = comments_df['description'].fillna('').str.len()
print(f'\nDescription length statistics:')
print(comments_df['desc_length'].describe())

# Sample comments
print(f'\nSample comments (shortest):')
display(comments_df.nsmallest(5, 'desc_length')[['order_id', 'description', 'desc_length']])

print(f'\nSample comments (longest):')
display(comments_df.nlargest(5, 'desc_length')[['order_id', 'description', 'desc_length']])

COMMENTS - Description Text Analysis
Null descriptions:  0
Empty descriptions: 0

Description length statistics:
count   93855.00
mean       79.11
std        77.05
min         1.00
25%        28.00
50%        56.00
75%       104.00
max       999.00
Name: desc_length, dtype: float64

Sample comments (shortest):


,order_id,description,desc_length
391,11189619,ه,1
2022,11239158,🌺,1
4322,11310012,🌺,1
6985,11405438,🤩,1
6993,11405593,🤩,1



Sample comments (longest):


,order_id,description,desc_length
15984,11760153,The same is often true for college application...,999
70542,13904162,شکلات داغ فاسد به تاریخ خرداد ۱۴۰۳ ارسال شده و...,995
70543,13904162,شکلات داغ فاسد به تاریخ خرداد ۱۴۰۳ ارسال شده و...,995
66857,13657114,با سلام و احترام،\nاین پیام جهت ثبت شکایت رسمی...,992
66858,13657114,با سلام و احترام،\nاین پیام جهت ثبت شکایت رسمی...,992


---
## 4️⃣ Cross-Table Relationship Summary

In [47]:
print('='*70)
print('RELATIONSHIP SUMMARY FOR SCHEMA DESIGN')
print('='*70)

print('''
┌─────────────────────────────────────────────────────────────────────┐
│                        TABLE RELATIONSHIPS                          │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   ORDERS (Parent)                                                   │
│   └── order_id (PK) ──────┬───────> CRM (Child)                     │
│                           │         └── order_id (FK, UNIQUE?)      │
│                           │                                         │
│                           └───────> COMMENTS (Child)                │
│                                     └── order_id (FK)               │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
''')

# Summary stats
print('\n📊 KEY FINDINGS:\n')

print(f'1. ORDERS table:')
print(f'   - {len(orders_df):,} records')
print(f'   - order_id is UNIQUE: {orders_df["order_id"].nunique() == len(orders_df)}')
print(f'   - {orders_df["user_id"].nunique():,} unique users')

print(f'\n2. CRM table:')
print(f'   - {len(crm_df):,} records')
print(f'   - order_id is UNIQUE: {crm_df["order_id"].nunique() == len(crm_df)}')
print(f'   - All order_ids in orders: {len(crm_order_ids - orders_order_ids) == 0}')

print(f'\n3. COMMENTS table:')
print(f'   - {len(comments_df):,} records')
print(f'   - {comments_df["order_id"].nunique():,} unique order_ids')
print(f'   - Orphan order_ids: {len(comments_not_in_orders):,}')

RELATIONSHIP SUMMARY FOR SCHEMA DESIGN

┌─────────────────────────────────────────────────────────────────────┐
│                        TABLE RELATIONSHIPS                          │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   ORDERS (Parent)                                                   │
│   └── order_id (PK) ──────┬───────> CRM (Child)                     │
│                           │         └── order_id (FK, UNIQUE?)      │
│                           │                                         │
│                           └───────> COMMENTS (Child)                │
│                                     └── order_id (FK)               │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘


📊 KEY FINDINGS:

1. ORDERS table:
   - 2,720,059 records
   - order_id is UNIQUE: True
   - 37

---
## 5️⃣ Data Quality Issues Summary

In [48]:
print('='*70)
print('DATA QUALITY ISSUES FOR SCHEMA DESIGN')
print('='*70)

issues = []

# Issue 1: is_otd has -1 values
if -1 in orders_df['is_otd'].values:
    count_minus1 = (orders_df['is_otd'] == -1).sum()
    issues.append(f'⚠️ is_otd has -1 values: {count_minus1:,} rows ({count_minus1/len(orders_df)*100:.2f}%)')

# Issue 2: Missing delivery_status
missing_status = orders_df['delivery_status'].isnull().sum()
if missing_status > 0:
    issues.append(f'⚠️ delivery_status has NULL: {missing_status:,} rows ({missing_status/len(orders_df)*100:.2f}%)')

# Issue 3: High missing rate for ratings
for col in ['rate_to_shop', 'rate_to_courier']:
    missing = crm_df[col].isnull().sum()
    pct = missing/len(crm_df)*100
    if pct > 50:
        issues.append(f'⚠️ {col} has {pct:.1f}% NULL values')

# Issue 4: Redundant order_date in CRM
issues.append(f'📝 CRM.order_date is redundant (same as orders.order_date)')

# Issue 5: Orphan comments
if len(comments_not_in_orders) > 0:
    issues.append(f'⚠️ {len(comments_not_in_orders):,} comment order_ids not in orders table')

for i, issue in enumerate(issues, 1):
    print(f'{i}. {issue}')

DATA QUALITY ISSUES FOR SCHEMA DESIGN
1. ⚠️ is_otd has -1 values: 165,936 rows (6.10%)
2. ⚠️ delivery_status has NULL: 44,014 rows (1.62%)
3. ⚠️ rate_to_shop has 58.6% NULL values
4. ⚠️ rate_to_courier has 73.7% NULL values
5. 📝 CRM.order_date is redundant (same as orders.order_date)
6. ⚠️ 5,037 comment order_ids not in orders table


---
## 6️⃣ Schema Design Recommendations

In [49]:
print('='*70)
print('SCHEMA DESIGN RECOMMENDATIONS')
print('='*70)

print('''
Based on EDA findings, here are the schema design recommendations:

┌─────────────────────────────────────────────────────────────────────┐
│ TABLE: orders                                                       │
├─────────────────────────────────────────────────────────────────────┤
│ - order_id       BIGINT       PRIMARY KEY                           │
│ - user_id        BIGINT       NOT NULL, INDEX                       │
│ - is_otd         SMALLINT     NULL (values: -1, 0, 1)               │
│ - order_date     DATE         NOT NULL, INDEX                       │
│ - delivery_status VARCHAR(50) NULL                                  │
└─────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────┐
│ TABLE: crm                                                          │
├─────────────────────────────────────────────────────────────────────┤
│ - order_id                        BIGINT  PK/UNIQUE, FK->orders     │
│ - crm_delivery_request_count      SMALLINT DEFAULT 0                │
│ - crm_fake_delivery_request_count SMALLINT DEFAULT 0                │
│ - rate_to_shop                    DECIMAL(2,1) NULL                 │
│ - rate_to_courier                 DECIMAL(2,1) NULL                 │
│                                                                     │
│ NOTE: Remove order_date (redundant with orders.order_date)          │
└─────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────┐
│ TABLE: comments                                                     │
├─────────────────────────────────────────────────────────────────────┤
│ - id             SERIAL       PRIMARY KEY (auto-increment)          │
│ - order_id       BIGINT       NOT NULL, INDEX                       │
│ - description    TEXT         NULL                                  │
│                                                                     │
│ NOTE: order_id NOT FK (orphans exist!)                              │
│       OR: FK with ON DELETE SET NULL / skip orphans during load     │
└─────────────────────────────────────────────────────────────────────┘

KEY DECISIONS TO MAKE:
1. Handle is_otd=-1: Keep as-is? Convert to NULL? Exclude?
2. Handle orphan comments: Skip during load? Allow without FK?
3. Handle missing ratings: Keep NULL? Default to some value?
''')

SCHEMA DESIGN RECOMMENDATIONS

Based on EDA findings, here are the schema design recommendations:

┌─────────────────────────────────────────────────────────────────────┐
│ TABLE: orders                                                       │
├─────────────────────────────────────────────────────────────────────┤
│ - order_id       BIGINT       PRIMARY KEY                           │
│ - user_id        BIGINT       NOT NULL, INDEX                       │
│ - is_otd         SMALLINT     NULL (values: -1, 0, 1)               │
│ - order_date     DATE         NOT NULL, INDEX                       │
│ - delivery_status VARCHAR(50) NULL                                  │
└─────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────┐
│ TABLE: crm                                                          │
├─────────────────────────────────────────────────────────────────────┤
│ - order_id                        

---
## 7️⃣ Additional Checks for Churn Definition

In [50]:
print('='*70)
print('TEMPORAL ANALYSIS FOR CHURN DEFINITION')
print('='*70)

# Convert to datetime
orders_df['order_date_dt'] = pd.to_datetime(orders_df['order_date'])

# Date range
min_date = orders_df['order_date_dt'].min()
max_date = orders_df['order_date_dt'].max()
date_range = (max_date - min_date).days

print(f'Date range: {min_date.date()} to {max_date.date()} ({date_range} days)')

# For churn: we need to define a "reference date" and look 30 days forward
# If max_date is the last date, we can only define churn for users whose last order
# was at least 30 days before max_date

cutoff_date = max_date - pd.Timedelta(days=30)
print(f'\nFor 30-day churn definition:')
print(f'  Max usable date for labeling: {cutoff_date.date()}')
print(f'  Users must have last order before this to be labeled')

# How many users have their last order before cutoff?
last_orders = orders_df.groupby('user_id')['order_date_dt'].max()
labelable_users = (last_orders <= cutoff_date).sum()
print(f'\n  Labelable users (last order <= {cutoff_date.date()}): {labelable_users:,}')
print(f'  Unlabelable users (last order after cutoff):           {len(last_orders) - labelable_users:,}')

TEMPORAL ANALYSIS FOR CHURN DEFINITION
Date range: 2025-03-16 to 2025-09-12 (180 days)

For 30-day churn definition:
  Max usable date for labeling: 2025-08-13
  Users must have last order before this to be labeled

  Labelable users (last order <= 2025-08-13): 219,801
  Unlabelable users (last order after cutoff):           156,197


In [51]:
# Orders per day distribution
print('='*70)
print('ORDERS PER DAY DISTRIBUTION')
print('='*70)

orders_per_day = orders_df.groupby('order_date_dt').size()
print(orders_per_day.describe())

print(f'\nDays with orders: {len(orders_per_day)}')
print(f'Days in range:    {date_range + 1}')
print(f'Missing days:     {date_range + 1 - len(orders_per_day)}')

ORDERS PER DAY DISTRIBUTION
count     180.00
mean    15111.44
std      4233.43
min      2858.00
25%     12770.50
50%     15668.50
75%     18104.25
max     23427.00
dtype: float64

Days with orders: 180
Days in range:    181
Missing days:     1


---
## ✅ FINAL SUMMARY

In [52]:
print('='*70)
print('FINAL EDA SUMMARY FOR SCHEMA DESIGN')
print('='*70)

summary = f'''
📊 DATASET SIZES:
- orders:   {len(orders_df):,} rows
- crm:      {len(crm_df):,} rows  
- comments: {len(comments_df):,} rows
- unique users: {orders_df['user_id'].nunique():,}

📅 DATE RANGE:
- {min_date.date()} to {max_date.date()} ({date_range} days)

🔑 PRIMARY KEYS:
- orders.order_id: UNIQUE = {orders_df['order_id'].nunique() == len(orders_df)}
- crm.order_id:    UNIQUE = {crm_df['order_id'].nunique() == len(crm_df)}
- comments: needs auto-increment ID (multiple comments per order possible: {(comments_df.groupby('order_id').size() > 1).sum() > 0})

🔗 FOREIGN KEY INTEGRITY:
- CRM->Orders:      All exist = {len(crm_order_ids - orders_order_ids) == 0}
- Comments->Orders: Orphans = {len(comments_not_in_orders):,}

⚠️ DATA QUALITY ISSUES:
1. is_otd values: {dict(orders_df['is_otd'].value_counts().sort_index())}
2. delivery_status NULL: {orders_df['delivery_status'].isnull().sum():,} ({orders_df['delivery_status'].isnull().sum()/len(orders_df)*100:.2f}%)
3. rate_to_shop NULL: {crm_df['rate_to_shop'].isnull().sum()/len(crm_df)*100:.1f}%
4. rate_to_courier NULL: {crm_df['rate_to_courier'].isnull().sum()/len(crm_df)*100:.1f}%
5. CRM.order_date is REDUNDANT: {same_dates == len(compare_df)}
6. Orphan comments: {len(comments_not_in_orders):,}

📝 RECOMMENDATIONS FOR SCHEMA:
1. orders.order_id -> BIGINT PRIMARY KEY
2. orders.is_otd -> SMALLINT (keep -1, 0, 1 as-is)
3. crm.order_id -> BIGINT PRIMARY KEY, FK to orders
4. crm: DROP order_date column (redundant)
5. comments: Add auto-increment ID as PK
6. comments.order_id: Either no FK (allow orphans) or filter during load
'''

print(summary)

FINAL EDA SUMMARY FOR SCHEMA DESIGN

📊 DATASET SIZES:
- orders:   2,720,059 rows
- crm:      2,720,059 rows  
- comments: 93,855 rows
- unique users: 375,998

📅 DATE RANGE:
- 2025-03-16 to 2025-09-12 (180 days)

🔑 PRIMARY KEYS:
- orders.order_id: UNIQUE = True
- crm.order_id:    UNIQUE = True
- comments: needs auto-increment ID (multiple comments per order possible: True)

🔗 FOREIGN KEY INTEGRITY:
- CRM->Orders:      All exist = True
- Comments->Orders: Orphans = 5,037

⚠️ DATA QUALITY ISSUES:
1. is_otd values: {-1: 165936, 0: 535039, 1: 2019084}
2. delivery_status NULL: 44,014 (1.62%)
3. rate_to_shop NULL: 58.6%
4. rate_to_courier NULL: 73.7%
5. CRM.order_date is REDUNDANT: True
6. Orphan comments: 5,037

📝 RECOMMENDATIONS FOR SCHEMA:
1. orders.order_id -> BIGINT PRIMARY KEY
2. orders.is_otd -> SMALLINT (keep -1, 0, 1 as-is)
3. crm.order_id -> BIGINT PRIMARY KEY, FK to orders
4. crm: DROP order_date column (redundant)
5. comments: Add auto-increment ID as PK
6. comments.order_id: Eith